In [ ]:
import pandas as pd
import random
from matplotlib import pyplot as plt


In [ ]:
all_players = pd.read_excel(r"C:\Users\ianto\Desktop\Progetto_fantacalcio.xlsx")
all_players = all_players.query("valido==1")

In [ ]:
all_players.head(10)

In [ ]:
def new_player(vincolo, df, ruolo, idscelti):
    if len(df.query("R == @ruolo and quotazione <= @vincolo and Id not in @idscelti") ) > 0:
        giocatore = df.query("R == @ruolo and quotazione <= @vincolo and Id not in @idscelti").sample(1).values.tolist()[0]
    else:
        giocatore = None
    return giocatore

In [ ]:
def new_rosa(budget):
    rosa = []
    idscelti = []
    giocatore = new_player(vincolo=budget,df=all_players, ruolo="P", idscelti = idscelti)
    rosa.append(giocatore)
    budget  = budget-giocatore[4]
    
    giocatore = new_player(vincolo=budget,df=all_players, ruolo="D", idscelti = idscelti)
    rosa.append(giocatore)
    idscelti.append(giocatore[0])
    budget  = budget-giocatore[4]
    
    lista_ruoli = ['A','C','D']
    giocatori_restanti = 9
    for i in range(3):
        for ruolo in lista_ruoli:
            giocatore = new_player(vincolo=budget-giocatori_restanti+1, df=all_players, ruolo=ruolo, idscelti=idscelti)
            rosa.append(giocatore)
            idscelti.append(giocatore[0])
            budget  = budget-giocatore[4]
            giocatori_restanti -= 1

    return rosa

In [ ]:
def fm_rosa(rosa):
    tot = 0
    for i in range(len(rosa)):
        tot += rosa[i][5]
    return tot

In [ ]:
def quotazione_rosa(rosa):
    tot = 0
    for i in range(len(rosa)):
        tot += rosa[i][4]
    return tot

In [ ]:
def crea_nuove_rose(numero, budget):
    rose = []
    for i in range(numero):
        rosa = new_rosa(budget)
        rose.append({"rosa":rosa,"fm":fm_rosa(rosa),"quotazione":quotazione_rosa(rosa)})
    return rose

In [ ]:
def estrai_migliori_rose(rose, numero_rose_migliori):
    rose_df = pd.DataFrame.from_dict(rose)
    rose_df["Rank"] = rose_df["fm"].transform(lambda x: x.rank(method = 'first', ascending=False))
    rose_df = rose_df.query("Rank <= @numero_rose_migliori").sort_values(by="Rank")[["rosa","fm","quotazione"]]
    rose_migliori = rose_df.to_dict(orient='records')
    return rose_migliori

In [ ]:
def new_rosa_combinata(rosa1, rosa2, budget):
    rosa_combinata = rosa1 + rosa2
    id_combinati = []
    for i in range(len(rosa_combinata)):
        id_combinati.append(rosa_combinata[i][0])
    id_combinati = list(set(id_combinati))
    df_filter = all_players.query("Id in @id_combinati")
    
    rosa = []
    idscelti = []
    giocatore = new_player(vincolo=budget,df=df_filter, ruolo="P",idscelti = idscelti )
    rosa.append(giocatore)
    budget  = budget-giocatore[4]
    
    giocatore = new_player(vincolo=budget,df=df_filter, ruolo="D",idscelti = idscelti )
    rosa.append(giocatore)
    idscelti.append(giocatore[0])
    budget  = budget-giocatore[4]
    
    lista_ruoli = ['A','C','D']
    giocatori_restanti = 9
    for i in range(3):
        for ruolo in lista_ruoli:
            giocatore = new_player(vincolo=budget-giocatori_restanti+1, df=df_filter, ruolo=ruolo, idscelti=idscelti)
            if giocatore is None:
                giocatore = new_player(vincolo=budget-giocatori_restanti+1, df=all_players, ruolo=ruolo, idscelti=idscelti)
            rosa.append(giocatore)
            idscelti.append(giocatore[0])
            budget  = budget-giocatore[4]
            giocatori_restanti -= 1

    return rosa

In [ ]:
def main(budget, numero_generazioni = 100, numero_rose_per_generazione = 200):
    best_fm = [0]
    numero_rose_migliori = int(numero_rose_per_generazione/2)
    numero_figli = int(numero_rose_per_generazione/4)
    generazioni=[]

    #creazione nuove rose random
    rose = crea_nuove_rose(numero = numero_rose_per_generazione, budget = budget)
    generazioni.append(rose)

    for i in range(numero_generazioni):

        #estrazione rose migliori dall'ultima generazione
        rose_migliori = estrai_migliori_rose(rose=generazioni[i],numero_rose_migliori=numero_rose_migliori)

        
        for j in range(numero_figli):
            k = random.randint(0, numero_rose_migliori-1)
            l = random.randint(0, numero_rose_migliori-1)
            if k==l and k!=0:
                l=0
            else:
                l=1
            rosa_combinata = new_rosa_combinata(rosa1 = rose_migliori[k]["rosa"], rosa2 = rose_migliori[l]["rosa"], budget = budget)
            rose_migliori.append({"rosa":rosa_combinata,"fm":fm_rosa(rosa_combinata),"quotazione":quotazione_rosa(rosa_combinata)})
        
        rose = crea_nuove_rose(numero_rose_per_generazione-len(rose_migliori), budget = budget)
        
        generazioni.append(rose_migliori + rose)
        best_fm.append(rose_migliori[0]["fm"])

        if i % 10 == 0:
            print(i)

    return generazioni,best_fm

In [ ]:
generazioni,best_fm = main(budget=209,numero_generazioni=50, numero_rose_per_generazione = 400)

In [ ]:
generazioni[50][0]

In [ ]:
plt.plot(best_fm)